In [0]:
GIT_USERNAME = 'AxelDasaev'
GIT_EMAIL = 'axeldasaev@gmail.com'
GIT_TOKEN = '3f9c42b6e2fcee4bdaadbf919a15e7ddab360eb1'
GIT_REPOSITORY = 'AxelDasaev/DeepDamperAxel'
GIT_PATH = 'https://github.com/AxelDasaev/DeepDamperAxel.git'

In [0]:
cd /content


/content


In [0]:
! pwd

/content


In [0]:
! ls

DeepDamperAxel	drive  sample_data


In [0]:
! git clone '{GIT_PATH}'

Cloning into 'DeepDamperAxel'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 113 (delta 63), reused 65 (delta 34), pack-reused 0
Receiving objects: 100% (113/113), 1.26 MiB | 3.61 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [0]:
%cd DeepDamperAxel

/content/DeepDamperAxel


In [0]:
!git config --global user.email '{GIT_EMAIL}'
!git config --global user.name '{GIT_USERNAME}'
!git remote rm origin
!git remote add origin https://'{GIT_TOKEN}'@github.com/'{GIT_REPOSITORY}'.git

In [4]:
%tensorflow_version 1.x

UsageError: Line magic function `%tensorflow_version` not found.


In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import os

In [2]:
print(tf.__version__)

1.15.0


In [3]:
def read_json(name):
    #print(name)
    with open(name) as data_file:    
            data = json.load(data_file)
    return data

In [4]:
def data_proces(name,Deface=100,NumSam=200):
    datos=read_json(name)
    entradas=read_json('_'.join(('inp',name)))
    train=np.array([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
    referencia=[0]
    for key in datos.keys():
        #print(len(datos[key]))
        for k,exp in enumerate(datos[key]):
            VelC=exp["velocity"]
            forceC=exp["force"]
            VelCj=[round(elem, 1) for elem in VelC]
            Conj=set(VelCj)
            infbar=100
            for elem in list(Conj):
                indices=[i for i,x in enumerate(VelCj) if x==elem]
                for ind in range(len(indices)-1):
                    if forceC[indices[ind]] == forceC[indices[ind+1]] and indices[ind]> 120:
                        reference = [indices[ind],indices[ind+1]]
                        break
                    elif abs(forceC[indices[ind]]-forceC[indices[ind+1]])<infbar and indices[ind+1]-indices[ind]>20 and indices[ind+1]-indices[ind]<200 and indices[ind]> 120:
                        tempref = [indices[ind],indices[ind+1]]
                        infbar = abs(forceC[indices[ind]]-forceC[indices[ind+1]])
                if 'reference' in locals():
                    break
            
            if 'reference' in locals():    
                Deface = reference[0]
                NumSam = reference[1] - reference[0]
                del reference
            else:
                Deface = tempref[0]
                NumSam = tempref[1] - tempref[0]
                del tempref
            referencia.append(referencia[-1]+NumSam+1)
            for element in range(NumSam+1):
                caso=list()
                caso.append(entradas[key]["Ap"])
                caso.append(entradas[key]["Ag"])
                caso.append(entradas[key]["h"])
                caso.append(entradas[key]["L"])
                caso.append(entradas[key]["viscocidad"])
                caso.append(entradas[key]["Ty"][k]/1000)
                caso.append((exp["velocity"][element+Deface])*0.001)
                caso.append(exp["displace"][element+Deface])
                caso.append(exp["force"][element+Deface])
                casonp=np.array(caso)
                #print(len(casonp))
                train=np.append(train,[casonp],axis=0)
            
    return train[2:][:],referencia


In [5]:
train,refer=data_proces("twofluidexp.json",Deface=120,NumSam=260)
#rint(train.shape)
#print(train[1:4])
np.random.shuffle(train)
#train[1:4,8:9].shape
#print(train[20][])
#print(refer)

In [6]:
Train=np.array(train)[:,:8].astype("float32")
Target=np.array(train)[:,8:9].astype("float32")

In [7]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Arquitectura de la red usando un modelo secuencial de Keras 

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

def DeepDamper ():
    model = tf.keras.Sequential([
    keras.layers.Dense(15,input_shape=(8,) , activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1)])
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
                  loss=tf.keras.losses.MeanSquaredError())
    return model

In [9]:
DamperModel = DeepDamper()
DamperModel.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                135       
_________________________________________________________________
dense_1 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 391
Trainable params: 391
Non-trainable params: 0
_________________________________________________________________


In [10]:
DamperModel.fit(x=Train, y=Target,batch_size =20, epochs=10000,callbacks=[cp_callback])

Train on 13333 samples
Epoch 1/10000
11960/13333 [=========================>....] - ETA: 0s - loss: 122.3114
Epoch 00001: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 59us/sample - loss: 119.3910
Epoch 2/10000
12300/13333 [==========================>...] - ETA: 0s - loss: 92.6328
Epoch 00002: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 92.6589
Epoch 3/10000
12500/13333 [===========================>..] - ETA: 0s - loss: 91.6645
Epoch 00003: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 91.4439
Epoch 4/10000
12860/13333 [===========================>..] - ETA: 0s - loss: 91.0173
Epoch 00004: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 42us/sample - loss: 91.0539
Epoch 5/10000
12220/13333 [==========================>...] - ETA: 0s - loss: 91.0354
Epoch 00005: saving model to training_1/cp.ckp

12420/13333 [==========================>...] - ETA: 0s - loss: 88.8537
Epoch 00039: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 88.6219
Epoch 40/10000
12500/13333 [===========================>..] - ETA: 0s - loss: 88.5979
Epoch 00040: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 88.5265
Epoch 41/10000
12520/13333 [===========================>..] - ETA: 0s - loss: 88.2112
Epoch 00041: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 88.3988
Epoch 42/10000
12300/13333 [==========================>...] - ETA: 0s - loss: 88.0733 ETA: 0s - loss: 87.92
Epoch 00042: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 88.2686
Epoch 43/10000
12680/13333 [===========================>..] - ETA: 0s - loss: 88.2045
Epoch 00043: saving model to training_1/cp.ckpt
13333/13333

12420/13333 [==========================>...] - ETA: 0s - loss: 77.3700
Epoch 00077: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 77.1347
Epoch 78/10000
12240/13333 [==========================>...] - ETA: 0s - loss: 76.6313
Epoch 00078: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 76.4698
Epoch 79/10000
12300/13333 [==========================>...] - ETA: 0s - loss: 75.6643
Epoch 00079: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 75.7537
Epoch 80/10000
12360/13333 [==========================>...] - ETA: 0s - loss: 75.1433
Epoch 00080: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 75.0417
Epoch 81/10000
12540/13333 [===========================>..] - ETA: 0s - loss: 74.2962
Epoch 00081: saving model to training_1/cp.ckpt
13333/13333 [====================

Epoch 116/10000
13100/13333 [============================>.] - ETA: 0s - loss: 30.8569
Epoch 00116: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 30.7776
Epoch 117/10000
12180/13333 [==========================>...] - ETA: 0s - loss: 29.2721
Epoch 00117: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 29.2471
Epoch 118/10000
12320/13333 [==========================>...] - ETA: 0s - loss: 27.7073
Epoch 00118: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 27.6965
Epoch 119/10000
12380/13333 [==========================>...] - ETA: 0s - loss: 26.2533
Epoch 00119: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 26.1737
Epoch 120/10000
12420/13333 [==========================>...] - ETA: 0s - loss: 24.6514
Epoch 00120: saving model to training_1/cp.ckpt
13333/13333 [

12320/13333 [==========================>...] - ETA: 0s - loss: 4.4068
Epoch 00154: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 4.4274
Epoch 155/10000
12360/13333 [==========================>...] - ETA: 0s - loss: 4.4081
Epoch 00155: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 4.4029
Epoch 156/10000
12140/13333 [==========================>...] - ETA: 0s - loss: 4.3419
Epoch 00156: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 4.4125
Epoch 157/10000
12520/13333 [===========================>..] - ETA: 0s - loss: 4.3819
Epoch 00157: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 39us/sample - loss: 4.3686
Epoch 158/10000
12340/13333 [==========================>...] - ETA: 0s - loss: 4.2938
Epoch 00158: saving model to training_1/cp.ckpt
13333/13333 [=========================

...

12360/13333 [==========================>...] - ETA: 0s - loss: 1.0229
Epoch 09814: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0307
Epoch 9815/10000
12440/13333 [==========================>...] - ETA: 0s - loss: 1.0346
Epoch 09815: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0302
Epoch 9816/10000
12360/13333 [==========================>...] - ETA: 0s - loss: 1.0359- ETA: 0s - loss: 1.
Epoch 09816: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0350
Epoch 9817/10000
12440/13333 [==========================>...] - ETA: 0s - loss: 1.0283- ETA: 0s - loss
Epoch 09817: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0247
Epoch 9818/10000
12320/13333 [==========================>...] - ETA: 0s - loss: 1.0381
Epoch 09818: saving model to training_1/cp.ckp

12260/13333 [==========================>...] - ETA: 0s - loss: 1.0262
Epoch 09852: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0222
Epoch 9853/10000
12360/13333 [==========================>...] - ETA: 0s - loss: 1.0223
Epoch 09853: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0259
Epoch 9854/10000
12380/13333 [==========================>...] - ETA: 0s - loss: 1.0251
Epoch 09854: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0275
Epoch 9855/10000
12160/13333 [==========================>...] - ETA: 0s - loss: 1.0385
Epoch 09855: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 42us/sample - loss: 1.0374
Epoch 9856/10000
12400/13333 [==========================>...] - ETA: 0s - loss: 1.0159
Epoch 09856: saving model to training_1/cp.ckpt
13333/13333 [=====================

13333/13333 [==============================] - 1s 42us/sample - loss: 1.0271
Epoch 9891/10000
12960/13333 [============================>.] - ETA: 0s - loss: 1.0337
Epoch 09891: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 43us/sample - loss: 1.0331
Epoch 9892/10000
13200/13333 [============================>.] - ETA: 0s - loss: 1.0271
Epoch 09892: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 42us/sample - loss: 1.0258
Epoch 9893/10000
13280/13333 [============================>.] - ETA: 0s - loss: 1.0283
Epoch 09893: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 42us/sample - loss: 1.0280
Epoch 9894/10000
12880/13333 [===========================>..] - ETA: 0s - loss: 1.0203
Epoch 09894: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 43us/sample - loss: 1.0215
Epoch 9895/10000
13320/13333 [============================>.] - ETA: 0s - l

Epoch 9929/10000
12420/13333 [==========================>...] - ETA: 0s - loss: 1.0179
Epoch 09929: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0216
Epoch 9930/10000
12340/13333 [==========================>...] - ETA: 0s - loss: 1.0381
Epoch 09930: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0383
Epoch 9931/10000
12120/13333 [==========================>...] - ETA: 0s - loss: 1.0360- ETA: 0s - loss
Epoch 09931: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 42us/sample - loss: 1.0336
Epoch 9932/10000
13320/13333 [============================>.] - ETA: 0s - loss: 1.0270
Epoch 09932: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 42us/sample - loss: 1.0274
Epoch 9933/10000
12200/13333 [==========================>...] - ETA: 0s - loss: 1.0281
Epoch 09933: saving model to training_1/cp.ckpt
1

12440/13333 [==========================>...] - ETA: 0s - loss: 1.0317
Epoch 09967: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 1.0282
Epoch 9968/10000
12280/13333 [==========================>...] - ETA: 0s - loss: 1.0183- ETA: 0s - loss: 
Epoch 09968: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 1.0210
Epoch 9969/10000
12280/13333 [==========================>...] - ETA: 0s - loss: 1.0334
Epoch 09969: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 41us/sample - loss: 1.0324
Epoch 9970/10000
12220/13333 [==========================>...] - ETA: 0s - loss: 1.0353
Epoch 09970: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 1s 40us/sample - loss: 1.0312
Epoch 9971/10000
12460/13333 [===========================>..] - ETA: 0s - loss: 1.0215
Epoch 09971: saving model to training_1/cp.ckpt
13333/13333 [===

In [16]:
DamperModel.save('my_net') 

OSError: Unable to create file (unable to open file: name = 'my_net', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 302)

In [17]:
DamperModel.save('my_net.h5') 

In [18]:
new_model = keras.models.load_model('my_net.h5')
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                135       
_________________________________________________________________
dense_1 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 391
Trainable params: 391
Non-trainable params: 0
_________________________________________________________________


In [14]:
keras.experimental.export_saved_model(DamperModel, 'my_net_SavedModel')

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.


AssertionError: Export directory already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory: my_net_SavedModel

In [15]:
new_model_SavedModel = keras.experimental.load_from_saved_model('my_net_SavedModel')
new_model_SavedModel.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 15)                135       
_________________________________________________________________
dense_4 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 16        
Total params: 391
Trainable params: 391
Non-trainable params: 0
_________________________________________________________________


In [0]:
#json_config = DamperModel.to_json()
#reinitialized_model = keras.models.model_from_json(json_config)
json_config = DamperModel.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)

DamperModel.save_weights('weights.h5')

with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('weights.h5')
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 15)                135       
_________________________________________________________________
dense_4 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 16        
Total params: 391
Trainable params: 391
Non-trainable params: 0
_________________________________________________________________


In [0]:
!git add -A
!git commit -m "Se agrego entrenamiento 1"
!git push -u origin master

[master 64feb36] Se agrego entrenamiento 1
 8 files changed, 4 insertions(+), 1 deletion(-)
 create mode 100644 my_net_SavedModel/assets/saved_model.json
 create mode 100644 my_net_SavedModel/saved_model.pb
 create mode 100644 my_net_SavedModel/variables/checkpoint
 create mode 100644 my_net_SavedModel/variables/variables.data-00000-of-00002
 create mode 100644 my_net_SavedModel/variables/variables.data-00001-of-00002
 create mode 100644 my_net_SavedModel/variables/variables.index
Counting objects: 12, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (11/11), done.
Writing objects: 100% (12/12), 16.92 KiB | 4.23 MiB/s, done.
Total 12 (delta 4), reused 1 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 3 local objects.
To https://github.com/AxelDasaev/DeepDamperAxel.git
   03046b3..64feb36  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
